<a href="https://colab.research.google.com/github/Kwannn666/GAN_AI/blob/main/Q36134255_HW7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 成大在地美食小幫手

實作一個成大附近必吃的小吃以及餐廳的對話RAG系統，

RAG資料集中裡面含有25家店的

地址，推薦菜色，價位，特色

###1. 讀入網址

將 Google Drive 上的檔案下載網址儲存成變數 URL ，並且將檔案下載並命名為 faiss_db.zip ， 最後列出壓縮檔案內的內容，以確認下載成功與檔案結構

這裡原本是按照教授在上課時輸入給 AI 生成程式碼指令的方式使用內建 AI 程式碼，但產生的結果都怪怪的，所裡後來使用 chatgpt 協助生成程式碼

In [ ]:
# 將網址儲存成變數
URL="https://drive.google.com/uc?export=download&id=1MdxblaAPaF19PRI86kbclHstk3LRHNBQ"

# 使用 wget 下載檔案，並儲存成 faiss_db.zip
!wget -O faiss_db.zip "$URL"

# 顯示下載內容（如果是文字檔案才可用 cat）
!unzip -l faiss_db.zip   # 列出壓縮檔內容


--2025-04-21 14:45:56--  https://drive.google.com/uc?export=download&id=1MdxblaAPaF19PRI86kbclHstk3LRHNBQ
Resolving drive.google.com (drive.google.com)... 108.177.12.100, 108.177.12.101, 108.177.12.113, ...
Connecting to drive.google.com (drive.google.com)|108.177.12.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1MdxblaAPaF19PRI86kbclHstk3LRHNBQ&export=download [following]
--2025-04-21 14:45:56--  https://drive.usercontent.google.com/download?id=1MdxblaAPaF19PRI86kbclHstk3LRHNBQ&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.134.132, 2607:f8b0:400c:c00::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.134.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11957 (12K) [application/octet-stream]
Saving to: ‘faiss_db.zip’

faiss_db.zip        100%[===================>]  11.68K  --.-KB/s    i

In [ ]:
!unzip faiss_db.zip

Archive:  faiss_db.zip
   creating: faiss_db/
  inflating: faiss_db/index.pkl      
  inflating: faiss_db/index.faiss    


### 2. 安裝並引入必要套件

引入標準套件

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
!pip install -U langchain langchain-community sentence-transformers faiss-cpu gradio openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.7/345.7 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 116.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/

引入 RAG（檢索增強生成）系統所需的主要工具套件

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from openai import OpenAI
import gradio as gr

### 3. 自訂 E5 embedding 類別

In [ ]:
class CustomE5Embedding(HuggingFaceEmbeddings):
    def embed_documents(self, texts):
        texts = [f"passage: {t}" for t in texts]
        return super().embed_documents(texts)

    def embed_query(self, text):
        return super().embed_query(f"query: {text}")

### 4. 載入 `faiss_db`

再度把Model讀入進來

In [ ]:
embedding_model = CustomE5Embedding(model_name="intfloat/multilingual-e5-small")
db = FAISS.load_local("faiss_db", embedding_model, allow_dangerous_deserialization=True)
retriever = db.as_retriever()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/498k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

### 5. 設定 LLM

在這邊我使用OpenAI的金鑰 ( 而非Groq ) ，並且使用的是模型是 gpt-4o

In [ ]:
import os
from google.colab import userdata
from openai import OpenAI

# 讀取 OpenAI API 金鑰
api_key = userdata.get('OpenAI')
os.environ["OPENAI_API_KEY"] = api_key

# 建立 OpenAI 客戶端
client = OpenAI(api_key=api_key)

# 指定模型
model = "gpt-4o"


### 6. prompt 設計

設定 system_prompt 為「成大在地美食小幫手」，回答語氣自然親切，符合學生期待，並且鼓勵使用具體的店名與餐點資訊，有效提升回覆的實用性與真實感

若資料不足，自然引導 GPT 說出「目前資料中查不到」並提供其他建議（如詢問同學或用 Google Maps）

In [ ]:
system_prompt = (
    "你是成大的美食推薦小幫手，熟悉校園周邊的各種小吃、餐廳和學生口袋名單。"
    "請用親切自然、像朋友一樣的語氣回覆同學的提問，推薦時盡量具體（包含店名、餐點、價位範圍）。"
    "如果資料中沒有找到答案，也可以誠實說明，並鼓勵使用者問問學長姐或用 Google Maps 幫忙查。"
    "回答時請用台灣人習慣的中文以及口吻、簡潔有溫度。"
)

prompt_template = """
請根據下列成大周邊美食資料，回答使用者的問題：
{retrieved_chunks}

使用者問的是：{question}

請根據提供的美食資料來回覆。如果資料中沒有提到，可以告訴使用者目前資料中查不到，也可以建議他參考 Google Maps 或問問在地同學推薦。
"""


### 7. 使用 RAG 來回應

搜尋與使用者問題相關的資訊，根據我們的 prompt 樣版去讓 LLM 回應。

In [ ]:
chat_history = []

def chat_with_rag(user_input):
    global chat_history
    # 取回相關資料
    docs = retriever.get_relevant_documents(user_input)
    retrieved_chunks = "\n\n".join([doc.page_content for doc in docs])

    # 將自定 prompt 套入格式
    final_prompt = prompt_template.format(retrieved_chunks=retrieved_chunks, question=user_input)

    # 呼叫 OpenAI API
    response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": final_prompt},
    ]
    )
    answer = response.choices[0].message.content

    chat_history.append((user_input, answer))
    return answer

### 8. 用 Gradio 打造 Web App

使用 gr.Blocks() 建立了一個簡單的對話介面，用來與 RAG 問答系統互動，
另外改造成文字會顯示成上下對話氣泡格式以及開啟就有歡迎訊息的格式

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("# 成大在地美食小幫手")
    with gr.Row():
        chatbot = gr.Chatbot(label="🍽️ 聊天紀錄")
        with gr.Column(scale=1, min_width=100):
            clear_btn = gr.Button("🧹 清除對話")

    with gr.Row():
        msg = gr.Textbox(placeholder="今晚我想來點...", show_label=False)
        send_btn = gr.Button("🚀 送出", variant="primary")

    # 初始提示
    chatbot.value = [("系統", "👋 哈囉！成大在地美食小幫手，請問你今晚想吃什麼？")]
    def respond(message, chat_history_local):
        response = chat_with_rag(message)
        chat_history_local.append((message, response))
        return "", chat_history_local

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch(debug=True)


<ipython-input-20-350b934b5e08>:4: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="🍽️ 聊天紀錄")


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://3b4afd4620ae5fdc0e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://3b4afd4620ae5fdc0e.gradio.live
